In [79]:
import pandas as pd
import os
DAT_DIR = './dat/yearly/'

def get_data(f):
    yr = int(f.replace('.csv',''))
    df = pd.read_csv(DAT_DIR + f).iloc[:,1:]
    df['Year'] = yr
    return yr, df
    
dd = [get_data(f) for f in os.listdir(DAT_DIR)]
dd = {k:v for k,v in dd}

# Weekly

In [5]:
import pandas as pd
import os
DAT_DIR = './dat/weekly/'

df = pd.read_csv('./dat/weekly/2004/week1.csv')

In [6]:
def get_valid_files(pth):
    return filter(lambda x: x!= '.DS_Store', os.listdir(pth))

ll = []

for sub_folder in get_valid_files(DAT_DIR):
    cur_dir = os.path.join(DAT_DIR, sub_folder)
    for week_file_name in get_valid_files(cur_dir):
        df = pd.read_csv(os.path.join(cur_dir, week_file_name))
        df['Year'] = int(sub_folder)
        df['Week'] = int(week_file_name.replace('week','').replace('.csv',''))
        ll.append(df)
        
    print(f'Done with {cur_dir}.')

Done with ./dat/weekly/2013.
Done with ./dat/weekly/2014.
Done with ./dat/weekly/2015.
Done with ./dat/weekly/2012.
Done with ./dat/weekly/2008.
Done with ./dat/weekly/2001.
Done with ./dat/weekly/2006.
Done with ./dat/weekly/2007.
Done with ./dat/weekly/2000.
Done with ./dat/weekly/2009.
Done with ./dat/weekly/2017.
Done with ./dat/weekly/2010.
Done with ./dat/weekly/2019.
Done with ./dat/weekly/2018.
Done with ./dat/weekly/2011.
Done with ./dat/weekly/2016.
Done with ./dat/weekly/1999.
Done with ./dat/weekly/2005.
Done with ./dat/weekly/2002.
Done with ./dat/weekly/2003.
Done with ./dat/weekly/2004.


## EDA

In [7]:
df_all = pd.concat(ll)
df_all = df_all[~df_all[['Player','Year','Week']].duplicated()]
df_all = df_all.sort_values(['Player','Year','Week'])
df_all.reset_index(drop = True, inplace = True)
#df_all['Tm'] = df_all['Tm'].str.replace('SDG','LAC').str.replace('OAK','LAR').str.replace('LAR','LVR')

In [8]:
len(set(df_all['Player']))

3628

In [9]:
len(set(df_all['Pos']))

137

In [10]:
len(set(df_all['Tm']))

34

In [11]:
reload2019 = True
if reload2019:
    # 538: https://github.com/fivethirtyeight/nfl-elo-game
    df_games = pd.read_csv('https://projects.fivethirtyeight.com/nfl-api/2019/nfl_games_2019.csv')
    df_games.to_csv('./dat/scores/scores2019.csv', index = False)
else:
    df_games = pd.read_csv('./dat/scores/scores2019.csv')
    
df_games = pd.concat([df_games, pd.read_csv('./dat/scores/nfl_games.csv')])
df_games['date'] = pd.to_datetime(df_games['date'])
df_games['DayOfWeek'] = df_games['date'].dt.dayofweek
df_games = df_games.sort_values(['date','team1','team2']).reset_index(drop = True)

In [12]:
first_games = df_games.groupby('season')['date'].min().reset_index()

Interestingly the day of week of the first game of the season has varied quite a bit and fallen on every day of the week but Monday or Tuesday. After the Super Bowl Era began, it was mostly on a Sunday, though a few seasons started on Friday/Saturday. Although it [seems](https://en.wikipedia.org/wiki/Thursday_Night_Football) Thursday Night Football did not debut until 2006-2007 season, the introduction of the season starting on a Thursday actually coincides with the debut of the [NFL Kickoff Game](https://en.wikipedia.org/wiki/NFL_Kickoff_Game) during the 2002-2003 season. Thus, the date from that page is probably related to the *brand* "Thursday Night Football" not the actual debut of football occurring on a Thursday. In fact, football occurred on Thursdays all the way back to 1920.  Note that the Wednesday Kickoff Game during the 2012-2013 season was due to [a conflict with the 2012 DNC](https://en.wikipedia.org/wiki/2012_NFL_season#Other_highlights). 

Based on this quick analysis, we are going to define the "Football Week" as Wednesday through the following Tuesday. 

In [13]:
pd.options.display.max_columns = 999

df_ct = df_games.groupby(['season','DayOfWeek'])['date'].count().reset_index()
df_ct['date'] = 1
df_ct = df_ct.pivot(index = 'DayOfWeek', columns = 'season', values = 'date').fillna(0)
df_ct = pd.DataFrame(df_ct.values.astype(int), columns = df_ct.columns, index = df_ct.index)
df_ct

season,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
DayOfWeek,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,1,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,0,1,0,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,1,0,1,1,1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [14]:
df_ct.sum(axis=1)

DayOfWeek
0     70
1     16
2     15
3     96
4     34
5     89
6    100
dtype: int64

In [15]:
df_games.groupby('DayOfWeek')['season'].min()

DayOfWeek
0    1921
1    1924
2    1925
3    1920
4    1927
5    1920
6    1920
Name: season, dtype: int64

In [16]:
pd.to_datetime(first_games['date']).dt.dayofweek.value_counts()

6    58
3    17
5    12
4    12
2     1
Name: date, dtype: int64

In [17]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (16,9))
dates = first_games['date']
plt.plot(dates, dates.dt.dayofweek, marker = 'x', linestyle = '')
plt.yticks(ticks = [2, 3, 4, 5, 6], labels = ['Wed.', 'Thurs.', 'Fri.', 'Sat.', 'Sun.'])
plt.axvline(pd.to_datetime('1966-04'), color = 'black', label = 'Super Bowl Era')
plt.axvline(pd.to_datetime('2002-04'), color = 'grey', label = '2002-2003 Season')
plt.legend()
plt.tight_layout();

In [18]:
first_games['DayOfWeek'] = first_games['date'].dt.dayofweek
first_games['FirstDayOfSeason'] = first_games['date'] - pd.TimedeltaIndex(first_games['DayOfWeek'] - 2, unit = 'D')

In [19]:
df_games = df_games.merge(first_games[['season','FirstDayOfSeason']], on = 'season', validate = 'm:1')
df_games['DaysSinceFirstDayOfSeason'] = (df_games['date'] - df_games['FirstDayOfSeason']).dt.days

In [20]:
df_games['week'] = 1 + (df_games['DaysSinceFirstDayOfSeason'] // 7)

In [21]:
df_games['week'].value_counts().sort_index()

1      910
2     1007
3     1015
4     1017
5     1023
6     1018
7     1024
8     1015
9     1025
10    1037
11    1056
12    1031
13     964
14     913
15     796
16     703
17     518
18     208
19     147
20      68
21      19
22      27
Name: week, dtype: int64

As a few sanity checks:

* The seasons where there *is* a week 22 are from 1991 onward (albeit not consistently)
* The first game of the 2020-2021 season was on `'2020-09-30'`. This is a Thursday so the true first day of the season is `'2020-09-02'` (a Wednesday). And 22 weeks and 4 days later takes us to `'2020-02-07'`, Super Bowl Sunday. 

In [22]:
sorted(df_games[df_games['week'] == 22]['season'].unique())

[1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 2000,
 2001,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [23]:
pd.to_datetime('2020-09-03') + pd.Timedelta(days = 22 * 7 + 4)

Timestamp('2021-02-08 00:00:00')

# 

We were after a map from season, week and team to opposing team. That is simply the sub-dataframe of the above with the respective columns. In theory it should not have duplicate entries, but some of the older games have this. It turns out those games actually have differing data (scores in particular). In the end, we are building a model on the player data which does not go back this far anyway so we will simplify things here and just drop anything the 1930-1931 season.

In [24]:
df_vs = df_games[['season','week','team1','team2']]

df_vs[df_vs.duplicated()]

,season,week,team1,team2
684,1927,12,CHI,FYJ
813,1929,12,NYG,FYJ


In [25]:
df_vs = df_games[df_games['season']>=1930][['season','week','team1','team2','score1','score2']]

In [26]:
df_vs_cp = df_vs.copy()

tmp = df_vs_cp['team2'].copy()
df_vs_cp['team2'] = df_vs_cp['team1']
df_vs_cp['team1'] = tmp

tmp = df_vs_cp['score2'].copy()
df_vs_cp['score2'] = df_vs_cp['score1']
df_vs_cp['score1'] = tmp

df_vs = pd.concat([df_vs, df_vs_cp]).sort_values(['season','week','team1']).reset_index(drop = True)
df_vs.columns = ['Year','Week','Team1','Team2','Score1','Score2']

In [27]:
df_vs.head()

,Year,Week,Team1,Team2,Score1,Score2
0,1930,1,DET,TOR,13,6
1,1930,1,TOR,DET,6,13
2,1930,2,ARI,GB,0,14
3,1930,2,BKN,CHI,0,0
4,1930,2,CHI,BKN,0,0


# Team Level Scores

In [28]:
res = (df_all.groupby('Pos')['PPRFantasyPoints'].sum() / df_all['PPRFantasyPoints'].sum())
res = res.sort_values(ascending = False)
print(res.head(20) * 100)
print()
print(res.cumsum().head(20) * 100)

Pos
WR       38.922234
RB       24.766645
QB       18.032709
TE       12.929195
FB        2.419646
HB        1.731186
FL        0.192790
SE        0.145734
RB/K      0.123978
RB-K      0.059419
WR/K      0.046054
H-B       0.031729
RB/KR     0.029831
WR-K      0.029194
XR        0.027088
ZR        0.024531
Z         0.022923
RWR       0.021777
X         0.019903
LWR       0.016325
Name: PPRFantasyPoints, dtype: float64

Pos
WR       38.922234
RB       63.688880
QB       81.721588
TE       94.650783
FB       97.070429
HB       98.801615
FL       98.994405
SE       99.140139
RB/K     99.264117
RB-K     99.323536
WR/K     99.369589
H-B      99.401318
RB/KR    99.431149
WR-K     99.460343
XR       99.487432
ZR       99.511963
Z        99.534886
RWR      99.556663
X        99.576566
LWR      99.592891
Name: PPRFantasyPoints, dtype: float64


In [30]:
pos_map = {'FB':'RB',
           'HB':'RB',
           'FL':'WR',
           'SE':'WR',
           'RB/K':'RB',
           'RB-K':'RB',
           'WR/K':'WR',
           'H-B':'WR',
           'RB/KR':'RB',
           'WR-K':'WR',
           'XR':'WR',
           'ZR':'WR',
           'Z':'WR',
           'RWR':'WR',
           'X':'WR',
           'LWR':'WR'
          }
df_pos_map = pd.DataFrame([pos_map.keys(), pos_map.values()]).T
df_pos_map.columns = ['Pos','SimplifiedPos']

df_all = df_all.merge(df_pos_map, how = 'left', on = 'Pos', validate = 'm:1')
df_all.loc[df_all['SimplifiedPos'].isnull(),'SimplifiedPos'] = df_all.loc[df_all['SimplifiedPos'].isnull(),'Pos']
del df_all['Pos']
df_all.rename({'SimplifiedPos':'Pos'}, axis = 1, inplace = True)

In [31]:
res = (df_all.groupby('Pos')['PPRFantasyPoints'].sum() / df_all['PPRFantasyPoints'].sum())
res = res.sort_values(ascending = False)
print(res.head(20) * 100)
print()
print(res.cumsum().head(20) * 100)

Pos
WR       39.500282
RB       29.130704
QB       18.032709
TE       12.929195
P         0.015885
TE/F      0.014788
WR/RS     0.012902
WR-P      0.011872
WR/R      0.011571
WR/P      0.011287
FB/T      0.010900
RB/F      0.010634
KR/P      0.010472
LT        0.010044
TB        0.008644
TE/L      0.008609
K         0.007945
KR/W      0.007348
F         0.006700
FB/R      0.006503
Name: PPRFantasyPoints, dtype: float64

Pos
WR       39.500282
RB       68.630987
QB       86.663695
TE       99.592891
P        99.608776
TE/F     99.623564
WR/RS    99.636466
WR-P     99.648338
WR/R     99.659909
WR/P     99.671196
FB/T     99.682096
RB/F     99.692730
KR/P     99.703202
LT       99.713246
TB       99.721890
TE/L     99.730499
K        99.738444
KR/W     99.745792
F        99.752492
FB/R     99.758995
Name: PPRFantasyPoints, dtype: float64


In [32]:
pos_map = {'P':'K',
           'TE/F':'TE',
           'WR/RS':'WR',
           'WR-P':'WR',
           'WR/R':'WR',
           'WR/P':'WR',
           'FB/T':'RB',
           'RB/F':'RB',
           #'KR/P':
           'LT':'',
           'TB':'RB',
           'TE/L':'TE',
           'KR/W':'WR',
           #'F':
           'FB/R':'RB'
          }
df_pos_map = pd.DataFrame([pos_map.keys(), pos_map.values()]).T
df_pos_map.columns = ['Pos','SimplifiedPos']

df_all = df_all.merge(df_pos_map, how = 'left', on = 'Pos', validate = 'm:1')
df_all.loc[df_all['SimplifiedPos'].isnull(),'SimplifiedPos'] = df_all.loc[df_all['SimplifiedPos'].isnull(),'Pos']
del df_all['Pos']
df_all.rename({'SimplifiedPos':'Pos'}, axis = 1, inplace = True)

In [33]:
res = (df_all.groupby('Pos')['PPRFantasyPoints'].sum() / df_all['PPRFantasyPoints'].sum())
res = res.sort_values(ascending = False)
print(res.head(20) * 100)
print()
print(res.cumsum().head(20) * 100)

Pos
WR       39.555262
RB       29.167385
QB       18.032709
TE       12.952593
K         0.023830
KR/P      0.010472
          0.010044
F         0.006700
T         0.006434
HB-K      0.006052
RB-KR     0.005982
HB/K      0.005230
X-WR      0.004478
KR        0.004189
DT        0.004189
LB        0.004131
OLB       0.003900
RDE       0.003900
TE/W      0.003830
QB/W      0.003529
Name: PPRFantasyPoints, dtype: float64

Pos
WR       39.555262
RB       68.722648
QB       86.755356
TE       99.707949
K        99.731779
KR/P     99.742251
         99.752295
F        99.758995
T        99.765428
HB-K     99.771480
RB-KR    99.777463
HB/K     99.782693
X-WR     99.787171
KR       99.791360
DT       99.795549
LB       99.799679
OLB      99.803579
RDE      99.807479
TE/W     99.811309
QB/W     99.814838
Name: PPRFantasyPoints, dtype: float64
